In [1]:
import sys
import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

py_ver = sys.version
k_ver = K.__version__
tf_ver = tf.__version__

print("Using Python version " + str(py_ver))
print("Using Keras version " + str(k_ver))
print("Using TensorFlow version " + str(tf_ver))

import pandas as pd
import numpy as np
import pyblp
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from scipy.optimize import minimize
import scipy
from numba import jit, njit, prange
from numba.typed import List
import time
import multiprocessing as mp
import pickle
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

Using Python version 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
Using Keras version 2.6.0
Using TensorFlow version 2.6.0


# Load Data

In [2]:
# Read the data
df = pd.read_csv(r'../data/swissmetro/swissmetro.dat', sep='\t')
print(df.shape)

(10728, 28)


In [3]:
i = 1
name = "swissmetro/fold"+str(i)
database = db.Database(name, df)
# The following statement allows you to use the names of the variable as Python variable.
globals().update(database.variables)

# Set biogeme train and test database
database = db.Database(name, df)

# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)

# Define some dummy variables
SM_COST = SM_CO * ( GA == 0 )
TRAIN_COST = TRAIN_CO * ( GA == 0 )

CAR_AV_SP = DefineVariable ('CAR_AV_SP', CAR_AV * ( SP !=0 ), database)
TRAIN_AV_SP = DefineVariable ('TRAIN_AV_SP', TRAIN_AV * ( SP != 0 ), database)

# Rescale some data
TRAIN_TT_SCALED   = DefineVariable('TRAIN_TT_SCALED',   TRAIN_TT / 100.0, database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100, database)
SM_TT_SCALED      = DefineVariable('SM_TT_SCALED',      SM_TT / 100.0   , database)
SM_COST_SCALED    = DefineVariable('SM_COST_SCALED',    SM_COST / 100   , database)
CAR_TT_SCALED     = DefineVariable('CAR_TT_SCALED',     CAR_TT / 100    , database)
CAR_CO_SCALED     = DefineVariable('CAR_CO_SCALED',     CAR_CO / 100    , database)

In [4]:
df = database.data
df.reset_index(drop=True, inplace=True)
print(df.shape)

(6768, 36)


In [10]:
# fold_index = df.index.to_list()
# np.random.shuffle(fold_index)
# bins = round(len(df)/5)
# start = 0
# for i in range(5):
#     test_index = fold_index[start : start+bins]
#     start = start+bins
#     df_test = df[df.index.isin(test_index)]
#     df_train = df[~df.index.isin(test_index)]

for col in ['Const_SM', 'Const_TRAIN', 'Const_CAR']:
    df[col] = 1
    
col = ['CHOICE', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE', 'MALE', 'GA', 'ORIGIN', 'DEST']
temp = pd.get_dummies(df[col], prefix=col, columns=col)
print(temp.shape)
df = pd.concat([df.reset_index(drop=True), temp], axis=1)
print(df.shape)

df_train = df.sample(frac = 0.8)
df_test = df[~df.index.isin(df_train.index)] 
print(len(df_train), len(df_test))

(6768, 68)
(6768, 107)
5414 1354


In [11]:
database_train = db.Database(name, df_train)
database_test = db.Database(name, df_test)

In [18]:
df.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,DEST_14,DEST_15,DEST_17,DEST_18,DEST_19,DEST_20,DEST_21,DEST_22,DEST_23,DEST_25
0,2,0,1,1,1,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,1,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,1,1,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0
3,2,0,1,1,1,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0
4,2,0,1,1,1,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0


**Three alternatives:**
- *unknown: 0*
- train: 1 
- SM: 2
- car: 3

In [17]:
df_train.CHOICE.value_counts()

2    3274
3    1407
1     733
Name: CHOICE, dtype: int64

## Research that uses NN to solve discrete choice model

Performances of NN 
- Hensher and Ton (2000) found that although artificial neural network (ANN) forecasts individuals’ choices better than nested logit models (NLM), NLM predicts market share ratios slightly better.
  - Hensher, D.A.; Ton, T.T. A Comparison of the Predictive Potential of Artificial Neural Networks and Nested Logit Models for Commuter Mode Choice. Transp. Res. E-Log. 2000, 36, 155–172.
  
- Cantarella and Luca (2005) compared a multilayer feedforward neural network model with random utility models (RUMs), and they **could not determine** which model was better.  
  - Cantarella, G.E.; de Luca, S. Multilayer Feedforward Networks for Transportation Mode Choice Analysis: An Analysis and a Comparison with Random Utility Models. Transp. Res. Part C Emerg. Technol. 2005, 13, 121–155.
  
- Some research shows that AI models to **outperform RUMs** such as NLM and cross-nested logit model (CNLM) in predicting travel mode choice.
  - Nam D, Cho J. Deep Neural Network Design for Modeling Individual-Level Travel Mode Choice Behavior[J]. Sustainability, 2020, 12(18): 7481.
  - Omrani, H.; Charif, O.; Gerber, P.; Awasthi, A.; Trigano, P. Prediction of Individual Travel Mode with Evidential Neural Network Model. Transp. Res. Rec. J. Transp. Res. Board 2013, 2399, 1–8.
  - Zhang, Y.L.; Xie, Y.C. Travel Mode Choice Modeling with Support Vector Machines. Transp. Res. Rec. J. Transp. Res. Board 2008, 2076, 141–150.

Potential issues besides performances
- Overfitting 
- Interpretability
  - Wang S, Wang Q, Bailey N, et al. Deep neural networks for choice analysis: A statistical learning theory perspective[J]. Transportation Research Part B: Methodological, 2021, 148: 60-81.

## DNN

### Simple DNN Structure
- The model expects rows of data with 9 variables (the input_dim=9 argument)
- The first hidden layer has 12 nodes and uses the relu activation function.
- The second hidden layer has 8 nodes and uses the relu activation function.
- The third hidden layer has 8 nodes and uses the relu activation function.
- The output layer has one node and uses the softmax activation function.

### Same Features 
- 'Const_SM', 'Const_TRAIN', 'Const_CAR', 'TRAIN_TT_SCALED', 'TRAIN_COST_SCALED',  'SM_TT_SCALED', 'SM_COST_SCALED', 'CAR_TT_SCALED', 'CAR_CO_SCALED'

In [33]:
features = ['Const_SM', 'Const_TRAIN', 'Const_CAR', 'TRAIN_TT_SCALED', 'TRAIN_COST_SCALED',  'SM_TT_SCALED', 
            'SM_COST_SCALED', 'CAR_TT_SCALED', 'CAR_CO_SCALED']

outcome = ['CHOICE_1', 'CHOICE_2', 'CHOICE_3']

trainX = df_train[features].values
trainy = df_train[outcome].values
testX = df_test[features].values
testy = df_test[outcome].values

model = K.models.Sequential()
model = Sequential()
model.add(Dense(12, input_dim=len(features), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

# fit model
history = model.fit(trainX, trainy, epochs=10, verbose='auto')
# # evaluate the model
# _, train_obj = model.evaluate(trainX, trainy)
# _, test_obj = model.evaluate(testX, testy)
# print('Test: %.4f' % (test_obj))

predicty = pd.DataFrame(model.predict(testX), columns = outcome)
predicty['y_Predicted'] = predicty.idxmax(axis=1)
predicty['y_Predicted'].replace({'CHOICE_0':0, 'CHOICE_1':1, 'CHOICE_2':2, 'CHOICE_3':3}, inplace=True)

predicty['y_Actual'] = df_test['CHOICE'].values

confusion = confusion_matrix(predicty['y_Actual'], predicty['y_Predicted'], labels=[0,1,2,3])
print(confusion)

accuracy = np.diagonal(confusion).sum()/confusion.sum()
print('Accuracy in test data:', accuracy)

loglike = np.log(np.multiply(predicty[outcome].values, testy).sum(axis=1)).sum()
print('Log Likelihood in test data:', loglike)

Epoch 1/10
170/170 [==============================] - 1s 933us/step - loss: 1.0141 - accuracy: 0.5423
Epoch 2/10
170/170 [==============================] - 0s 960us/step - loss: 0.8731 - accuracy: 0.6051
Epoch 3/10
170/170 [==============================] - 0s 930us/step - loss: 0.8173 - accuracy: 0.6319
Epoch 4/10
170/170 [==============================] - 0s 968us/step - loss: 0.7780 - accuracy: 0.6481
Epoch 5/10
170/170 [==============================] - 0s 953us/step - loss: 0.7506 - accuracy: 0.6576
Epoch 6/10
170/170 [==============================] - 0s 960us/step - loss: 0.7368 - accuracy: 0.6638
Epoch 7/10
170/170 [==============================] - 0s 985us/step - loss: 0.7263 - accuracy: 0.6655
Epoch 8/10
170/170 [==============================] - 0s 977us/step - loss: 0.7230 - accuracy: 0.6690
Epoch 9/10
170/170 [==============================] - 0s 993us/step - loss: 0.7184 - accuracy: 0.6697
Epoch 10/10
170/170 [==============================] - 0s 1ms/step - loss: 0.7139 

### Simple DNN Structure
- The model expects rows of data with xx variables (the input_dim = xx argument)
- The first hidden layer has 12 nodes and uses the relu activation function.
- The second hidden layer has 8 nodes and uses the relu activation function.
- The third hidden layer has 8 nodes and uses the relu activation function.
- The output layer has one node and uses the softmax activation function.

### ALL Features 

In [32]:
features = []
for item in ['PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE', 'MALE', 'GA', 'ORIGIN', 'DEST',
            'Const_SM', 'Const_TRAIN', 'Const_CAR', 'TRAIN_TT_SCALED', 'TRAIN_COST_SCALED',  'SM_TT_SCALED', 
            'SM_COST_SCALED', 'CAR_TT_SCALED', 'CAR_CO_SCALED']:
    temp1 = df.columns[df.columns.str.contains(item)] 
    for name in temp1:
        features.append(name)

outcome = ['CHOICE_1', 'CHOICE_2', 'CHOICE_3']

trainX = df_train[features].values
trainy = df_train[outcome].values
testX = df_test[features].values
testy = df_test[outcome].values

model = K.models.Sequential()
model = Sequential()
model.add(Dense(12, input_dim=len(features), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

# fit model
history = model.fit(trainX, trainy, epochs=10, verbose='auto')
# # evaluate the model
# _, train_obj = model.evaluate(trainX, trainy)
# _, test_obj = model.evaluate(testX, testy)
# print('Test: %.4f' % (test_obj))

predicty = pd.DataFrame(model.predict(testX), columns = outcome)
predicty['y_Predicted'] = predicty.idxmax(axis=1)
predicty['y_Predicted'].replace({'CHOICE_0':0, 'CHOICE_1':1, 'CHOICE_2':2, 'CHOICE_3':3}, inplace=True)

predicty['y_Actual'] = df_test['CHOICE'].values

confusion = confusion_matrix(predicty['y_Actual'], predicty['y_Predicted'], labels=[0,1,2,3])
print(confusion)

accuracy = np.diagonal(confusion).sum()/confusion.sum()
print('Accuracy in test data:', accuracy)

loglike = np.log(np.multiply(predicty[outcome].values, testy).sum(axis=1)).sum()
print('Log Likelihood in test data:', loglike)

Epoch 1/10
170/170 [==============================] - 0s 962us/step - loss: 1.1251 - accuracy: 0.5405
Epoch 2/10
170/170 [==============================] - 0s 974us/step - loss: 0.8891 - accuracy: 0.6058
Epoch 3/10
170/170 [==============================] - 0s 1ms/step - loss: 0.8150 - accuracy: 0.6321
Epoch 4/10
170/170 [==============================] - 0s 975us/step - loss: 0.7614 - accuracy: 0.6583
Epoch 5/10
170/170 [==============================] - 0s 955us/step - loss: 0.7368 - accuracy: 0.6638
Epoch 6/10
170/170 [==============================] - 0s 1ms/step - loss: 0.7242 - accuracy: 0.6683
Epoch 7/10
170/170 [==============================] - 0s 1ms/step - loss: 0.7184 - accuracy: 0.6642
Epoch 8/10
170/170 [==============================] - 0s 1ms/step - loss: 0.7124 - accuracy: 0.6764
Epoch 9/10
170/170 [==============================] - 0s 992us/step - loss: 0.7099 - accuracy: 0.6690
Epoch 10/10
170/170 [==============================] - 0s 955us/step - loss: 0.7023 - accu

## More Delicate DNN Structure
- Input layer
- 1st hidden layer: Relu activate function; 80 perceptrons
- 2ed hidden layer: Relu activate function; 80 perceptrons
- 3rd hidden layer: Relu activate function; 80 perceptrons
- 4st hidden layer: Sigmoid activate function; 3 perceptrons

In [31]:
features = []
for item in ['PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE', 'MALE', 'GA', 'ORIGIN', 'DEST',
            'Const_SM', 'Const_TRAIN', 'Const_CAR', 'TRAIN_TT_SCALED', 'TRAIN_COST_SCALED',  'SM_TT_SCALED', 
            'SM_COST_SCALED', 'CAR_TT_SCALED', 'CAR_CO_SCALED']:
    temp1 = df.columns[df.columns.str.contains(item)] 
    for name in temp1:
        features.append(name)

outcome = ['CHOICE_1', 'CHOICE_2', 'CHOICE_3']

trainX = df_train[features].values
trainy = df_train[outcome].values
testX = df_test[features].values
testy = df_test[outcome].values

scalar = MinMaxScaler()
scalar.fit(df[features].values)
trainX = scalar.transform(trainX)
testX = scalar.transform(testX)

init = K.initializers.glorot_uniform(seed=1)
model = K.models.Sequential()
model = Sequential()
model.add(Dense(80, input_dim=len(features), activation='relu', kernel_initializer=init))
model.add(Dropout(0.55))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

# fit model
history = model.fit(trainX, trainy, epochs=500, verbose='auto', batch_size=100)
# # evaluate the model
# _, train_obj = model.evaluate(trainX, trainy)
# _, test_obj = model.evaluate(testX, testy)
# print('Test: %.4f' % (test_obj))

predicty = pd.DataFrame(model.predict(testX), columns = outcome)
predicty['y_Predicted'] = predicty.idxmax(axis=1)
predicty['y_Predicted'].replace({'CHOICE_0':0, 'CHOICE_1':1, 'CHOICE_2':2, 'CHOICE_3':3}, inplace=True)

predicty['y_Actual'] = df_test['CHOICE'].values

confusion = confusion_matrix(predicty['y_Actual'], predicty['y_Predicted'], labels=[0,1,2,3])
print(confusion)

accuracy = np.diagonal(confusion).sum()/confusion.sum()
print('Accuracy in test data:', accuracy)

loglike = np.log(np.multiply(predicty[outcome].values, testy).sum(axis=1)).sum()
print('Log Likelihood in test data:', loglike)

Epoch 1/500
55/55 [==============================] - 0s 1ms/step - loss: 1.0588 - accuracy: 0.5175
Epoch 2/500
55/55 [==============================] - 0s 1ms/step - loss: 0.9816 - accuracy: 0.6047
Epoch 3/500
55/55 [==============================] - 0s 1ms/step - loss: 0.9502 - accuracy: 0.6047
Epoch 4/500
55/55 [==============================] - 0s 2ms/step - loss: 0.9194 - accuracy: 0.6119
Epoch 5/500
55/55 [==============================] - 0s 1ms/step - loss: 0.8956 - accuracy: 0.6234
Epoch 6/500
55/55 [==============================] - 0s 1ms/step - loss: 0.8789 - accuracy: 0.6315
Epoch 7/500
55/55 [==============================] - 0s 1ms/step - loss: 0.8657 - accuracy: 0.6361
Epoch 8/500
55/55 [==============================] - 0s 2ms/step - loss: 0.8600 - accuracy: 0.6367
Epoch 9/500
55/55 [==============================] - 0s 2ms/step - loss: 0.8446 - accuracy: 0.6402
Epoch 10/500
55/55 [==============================] - 0s 2ms/step - loss: 0.8296 - accuracy: 0.6404
Epoch 11/

55/55 [==============================] - 0s 1ms/step - loss: 0.5475 - accuracy: 0.7588
Epoch 165/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5446 - accuracy: 0.7538
Epoch 166/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5429 - accuracy: 0.7593
Epoch 167/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5503 - accuracy: 0.7560
Epoch 168/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5463 - accuracy: 0.7540
Epoch 169/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5485 - accuracy: 0.7556
Epoch 170/500
55/55 [==============================] - 0s 2ms/step - loss: 0.5439 - accuracy: 0.7591
Epoch 171/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5389 - accuracy: 0.7615
Epoch 172/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5480 - accuracy: 0.7591
Epoch 173/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5493 - accuracy: 0.7590
Epoc

55/55 [==============================] - 0s 1ms/step - loss: 0.4936 - accuracy: 0.7782
Epoch 327/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4985 - accuracy: 0.7793
Epoch 328/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4933 - accuracy: 0.7776
Epoch 329/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4944 - accuracy: 0.7800
Epoch 330/500
55/55 [==============================] - 0s 1ms/step - loss: 0.5034 - accuracy: 0.7793
Epoch 331/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4984 - accuracy: 0.7854
Epoch 332/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4949 - accuracy: 0.7776
Epoch 333/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4932 - accuracy: 0.7767
Epoch 334/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4897 - accuracy: 0.7826
Epoch 335/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4932 - accuracy: 0.7795
Epoc

55/55 [==============================] - 0s 1ms/step - loss: 0.4759 - accuracy: 0.7822
Epoch 489/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4831 - accuracy: 0.7763
Epoch 490/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4728 - accuracy: 0.7846
Epoch 491/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4730 - accuracy: 0.7837
Epoch 492/500
55/55 [==============================] - 0s 2ms/step - loss: 0.4806 - accuracy: 0.7856
Epoch 493/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4720 - accuracy: 0.7918
Epoch 494/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4782 - accuracy: 0.7870
Epoch 495/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4831 - accuracy: 0.7820
Epoch 496/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4703 - accuracy: 0.7850
Epoch 497/500
55/55 [==============================] - 0s 1ms/step - loss: 0.4721 - accuracy: 0.7868
Epoc

Discrete Choice Model

In [30]:
trainX = df_train[features].values
trainy = df_train[outcome].values
testX = df_test[features].values
testy = df_test[outcome].values

In [14]:
# Create parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

# Definition of the utility functions
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3}

# Associate the availability conditions with the alternatives
av = {1: TRAIN_AV_SP, 2: SM_AV, 3: CAR_AV_SP}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme  = bio.BIOGEME(database_train, logprob)
biogeme.modelName = "swissmetro_logit_train"

# Train the model using the training set
biogeme.generateHtml = True
biogeme.generatePickle = True

results = biogeme.estimate()
print(f"HTML file:    {results.data.htmlFileName}")
print(f"Pickle file:  {results.data.pickleFileName }")

# Let's look at results 
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k:10}=\t{v:.3g}")
    
# Estimate the parameters
results = biogeme.estimate()



HTML file:    swissmetro_logit_train~17.html
Pickle file:  swissmetro_logit_train~17.pickle
ASC_CAR   =	-0.196
ASC_TRAIN =	-0.733
B_COST    =	-1.08
B_TIME    =	-1.22


In [16]:
# Look at test data
prob_train = models.logit(V, av, 1)
prob_SM = models.logit(V, av, 2)
prob_car = models.logit(V, av, 3)

simulate ={'Prob. SM':  prob_SM ,
           'Prob. train':  prob_train ,
           'Prob. car': prob_car ,}

biogeme = bio.BIOGEME(database_test, simulate)
biogeme.modelName = "swissmetro_logit_test"

betas = biogeme.freeBetaNames
results = res.bioResults(pickleFile='swissmetro_logit_train.pickle')
betaValues = results.getBetaValues()

simulatedValues = biogeme.simulate(betaValues)

prob_max = simulatedValues.idxmax(axis=1)
prob_max = prob_max.replace({'Prob. train': 1, 'Prob. SM': 2, 'Prob. car': 3})

data = {'y_Actual':    df_test['CHOICE'],
        'y_Predicted': prob_max}

predict_df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])

confusion = confusion_matrix(predict_df['y_Actual'], predict_df['y_Predicted'], labels=[0,1,2,3])
print(confusion)

accuracy = np.diagonal(confusion).sum()/confusion.sum()
print('Accuracy in test data:', accuracy)

loglike = np.log(np.multiply(simulatedValues[['Prob. train', 'Prob. SM', 'Prob. car']], testy).sum(axis=1)).sum()
print('Log Likelihood in test data:', loglike)

[[  0   0   0   0]
 [  0   1 165   9]
 [  0   0 757  59]
 [  0   0 205 158]]
Accuracy in test data: 0.6765140324963073
Log Likelihood in test data: -1049.8050399275562
